#### Importação das bibliotecas

In [32]:
import pprint
from gensim import corpora, models, similarities

#### Exemplo de Corpus (Coleção de Documentos)
##### Textos escritos por Shakespeare na wikipedia

In [3]:
text_corpus = [
    "Human machine interface for lab abc computer applications",
    "A survey of user opinion of computer system response time",
    "The EPS user interface management system",
    "System and human system engineering testing of EPS",
    "Relation of user perceived response time to error measurement",
    "The generation of random binary unordered trees",
    "The intersection graph of paths in trees",
    "Graph minors IV Widths of trees and well quasi ordering",
    "Graph minors A survey",
]

#### Processando o Corpus

In [7]:
# Criando conjunto de palabras frequêntes
stoplist = set('for a of the and to in'.split(' '))

# Convertendo cada documento para letras minúsculas, aplicando split em cada documento e filtrando palavras irrelevantes
texts =[[word for word in document.lower().split() if word not in stoplist]
       for document in text_corpus]

# Contando frequências de palavras
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1
        
# Mantendo apenas as palavras que aparecem mais de uma vez
processed_corpus = [[token for token in text if frequency[token] > 1] for text in texts]
pprint.pprint(processed_corpus)

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]


#### Associação de cada palavra do corpus com um unico id, formando assim um dicionário que define o vocabulário de todas as palavras que o processamento conhece

In [10]:
dictionary = corpora.Dictionary(processed_corpus)
print(dictionary)

Dictionary<12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...>


#### Dicionário das palavras com seus respectivos ids, gerados a partir da tokenização

In [22]:
pprint.pprint(dictionary.token2id)

{'computer': 0,
 'eps': 8,
 'graph': 10,
 'human': 1,
 'interface': 2,
 'minors': 11,
 'response': 3,
 'survey': 4,
 'system': 5,
 'time': 6,
 'trees': 9,
 'user': 7}


#### Exemplo utilizando a técnica bag-of-word

In [24]:
# Primeira entrada da tupla cooresponde ao id do token do dicionário 
# Segunda entrada da tupla cooresponde a quantidade de vezes que a palavra aparece na sentença
new_doc = "Human computer interaction"
new_vec = dictionary.doc2bow(new_doc.lower().split())
print(new_vec)

[(0, 1), (1, 1)]


#### Convertendo corpus de entrada original para lista de vetores utilizando técnica bag-of-word

In [25]:
bow_corpus = [dictionary.doc2bow(text) for text in processed_corpus]
pprint.pprint(bow_corpus)

[[(0, 1), (1, 1), (2, 1)],
 [(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)],
 [(2, 1), (5, 1), (7, 1), (8, 1)],
 [(1, 1), (5, 2), (8, 1)],
 [(3, 1), (6, 1), (7, 1)],
 [(9, 1)],
 [(9, 1), (10, 1)],
 [(9, 1), (10, 1), (11, 1)],
 [(4, 1), (10, 1), (11, 1)]]


#### Transformando o corpus depois de vetorizado utilizando o modelo tf-idf
##### O modelo tf-idf transforma vetores da representação bag-of-words em um espaço vetorial onde as contagens de frequências são ponderadas de acordo com a raridade relativa de cada palavra no corpus.
##### Palavras que aparecem mais vezes por exemplo "system" tem pesos menores do que palavras que aparecem menos vezes por exemplo "minors". Subtende-se que palavras que aparecem menos vezes tem mais representatividade.

In [31]:
# treinando o modelo
tfidf = models.TfidfModel(bow_corpus)

# transformando "system minors"
words = "system minors".lower().split()
print(tfidf[dictionary.doc2bow(words)])

[(5, 0.5898341626740045), (11, 0.8075244024440723)]


#### Transformando todo Corpus via TfIdf e indexando e preparando para consultas de similaridade:

In [33]:
index = similarities.SparseMatrixSimilarity(tfidf[bow_corpus], num_features=12)

#### Consultando a similaridade de um novo documento em relação a todos documentos do corpus

In [39]:
query_document = "system engineering".split()
query_bow = dictionary.doc2bow(query_document)
sims = index[tfidf[query_bow]]
print(list(enumerate(sims)))

[(0, 0.0), (1, 0.32448703), (2, 0.41707572), (3, 0.7184812), (4, 0.0), (5, 0.0), (6, 0.0), (7, 0.0), (8, 0.0)]


#### Visualizando scores de similaridade relacionando com respectivo documento

In [40]:
for document_number, score in sorted(enumerate(sims), key=lambda x: x[1], reverse=True):
    print(document_number, score)

3 0.7184812
2 0.41707572
1 0.32448703
0 0.0
4 0.0
5 0.0
6 0.0
7 0.0
8 0.0
